In [1]:
import tensorflow as tf

CNN Prototype

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

# This was copy-pasted from my old personal project.
# Some of the layers are not applicable (normalization),
# and some layers (input, reshape, resizing, and dense) need to be resized.

model = Sequential([
    layers.Input(shape = (81141,)),
    layers.Reshape((129, 629, 1)),
    layers.Resizing(height = 129, width = 129),
    layers.Normalization(mean = 126, variance = 165000),
    layers.Conv2D(4, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(4, 3, activation='relu'),
    layers.Flatten(),
    layers.Dense(200, activation='relu'),
    layers.Dense(160, activation='relu'),
    layers.Dense(120, activation='relu'),
    layers.Dense(80, activation='relu'),
    layers.Dense(37, activation='relu')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 129, 629, 1)       0         
_________________________________________________________________
resizing (Resizing)          (None, 129, 129, 1)       0         
_________________________________________________________________
normalization (Normalization (None, 129, 129, 1)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 127, 127, 4)       40        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 4)         148       
_________________________________________________________________
flatten (Flatten)            (None, 14884)             0

(this cell is just me experimenting with loading the data and training on it)

In [ ]:
save_to_disk = False

# TrainFeatures
# TrainTargets

model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['mean_squared_error', 'mean_absolute_error'])
model.fit(TrainFeatures,TrainTargets,epochs = 500, batch_size = 4)
if save_to_disk:
    # Saving model to JSON and weights to H5.
    os.chdir('..')
    os.chdir('./models/')
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("model.h5")
    print("Saved model to disk")
loss  = model.evaluate(ValFeatures, ValTargets)
print('Loss on Validation Set: ', loss)

RNN Prototype